In [ ]:
pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import metrics

In [ ]:
#loading the excel file into pandas dataframe and inspecting the first 5 rows
cars = pd.read_excel("jijiCarsRawDataFinal(1).xlsx")
cars.head()

In [ ]:
#checking the number of rows and columns
cars.shape